### Imports

In [1]:
# Libraries
import os
from importlib import reload

import tensorflow as tf
import keras
import sklearn.model_selection

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Scripts
from helpers import data, runners, utils, generators

In [2]:
!pip show tensorflow # Should be v2.10

Name: tensorflow
Version: 2.10.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


In [3]:
!pip show keras # Should be v2.10

Name: keras
Version: 2.10.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: c:\users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages
Requires: 
Required-by: keras-tuner, tensorflow


In [4]:
reload(runners)
reload(data)
reload(utils)

<module 'helpers.utils' from 'c:\\Users\\sonar\\Desktop\\thesis-ml\\ml\\main\\helpers\\utils.py'>

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Check whether tf is using the GPU
utils.check_gpu_use() 

GPU DETECTED ✓
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## `main`

In [6]:
NUM_EPOCHS = 50
DATA_BATCH_SIZE = 16
GPU_BATCH_SIZE = 4

### Load data

In [7]:
pairs_df = pd.read_csv("../data/DATASET_CSVS/deepship_pairs_diff_recording.csv")

print(pairs_df.shape)

(12880, 5)


In [8]:
# ADD RELATIVE PATH

PATH_TO_ROOT = "..\\data\\"
pairs_df["file_path_1"] = pairs_df["file_path_1"].apply(
    lambda x: PATH_TO_ROOT + x
)
pairs_df["file_path_2"] = pairs_df["file_path_2"].apply(
    lambda x: PATH_TO_ROOT + x
)

pairs_df.head(2)

,ship_name,file_path_1,date_seg_1,file_path_2,date_seg_2
0,ALFRED_N,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170326_1,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170403_51
1,ALFRED_N,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170326_2,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170403_48


In [9]:
# Get train-validate-test splits

train_df, test_df = sklearn.model_selection.train_test_split(
    pairs_df,
    test_size=0.2,
    random_state=100,
    shuffle=True
)

val_df, test_df = sklearn.model_selection.train_test_split(
    test_df,
    test_size=0.5,
    random_state=100,
    shuffle=True
)

In [10]:
# Initialise generators
train_gen = generators.N2NGenerator(
    train_df,
    'mat',
    'Ptrans',
    batch_size=DATA_BATCH_SIZE,
    shuffle=True,
    conv_channel=True
)

val_gen = generators.N2NGenerator(
    val_df,
    'mat',
    'Ptrans',
    batch_size=DATA_BATCH_SIZE,
    shuffle=True,
    conv_channel=True
)

test_gen = generators.N2NGenerator(
    test_df,
    'mat',
    'Ptrans',
    batch_size=DATA_BATCH_SIZE,
    shuffle=True,
    conv_channel=True
)

### Initialise model

In [11]:
# model = keras.saving.load_model('models/saved/n2n_finetuned_25112024/n2n_finetuned.keras')

from models import irfan_2020, unet_n2n, unetpro

# model = irfan_2020.get_irfan_model(input_shape=(192, 192, 1))
model = unet_n2n.get_unet_model(input_shape=(192, 192, 1))

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.99, epsilon=1e-8),
    loss=keras.losses.MeanSquaredError(),
    metrics=[utils.psnr]
)

model.summary()

Model: "unet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 192, 192, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 192, 192, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 192, 192, 64  0           ['conv2d[0][0]']                 
                                )                                                              

### Train model

In [ ]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="models/saved/diff_spec_denoiser_05122024/unet/{epoch:02d}.keras",
    monitor='val_psnr',
    mode='max',
    save_best_only=True
)

csv_logger = keras.callbacks.CSVLogger(
    filename="models/saved/diff_spec_denoiser_05122024/unet/training.log",
    separator=",",
    append=False
)

backup_callback = keras.callbacks.BackupAndRestore(
    backup_dir="models/saved/diff_spec_denoiser_05122024/unet/tmp"
)

: 

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    batch_size=GPU_BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[model_checkpoint_callback, csv_logger, backup_callback],
    verbose=2
)

Epoch 1/50
644/644 - 1010s - loss: 0.0273 - psnr: 16.1409 - val_loss: 0.0231 - val_psnr: 16.6583 - 1010s/epoch - 2s/step
Epoch 2/50
644/644 - 1232s - loss: 0.0233 - psnr: 16.6527 - val_loss: 0.0225 - val_psnr: 16.7866 - 1232s/epoch - 2s/step
Epoch 3/50
644/644 - 1364s - loss: 0.0226 - psnr: 16.7806 - val_loss: 0.0221 - val_psnr: 16.8391 - 1364s/epoch - 2s/step
Epoch 4/50
644/644 - 1211s - loss: 0.0222 - psnr: 16.8497 - val_loss: 0.0227 - val_psnr: 16.7307 - 1211s/epoch - 2s/step
Epoch 5/50
644/644 - 1089s - loss: 0.0220 - psnr: 16.8799 - val_loss: 0.0218 - val_psnr: 16.9186 - 1089s/epoch - 2s/step
Epoch 6/50
644/644 - 1104s - loss: 0.0218 - psnr: 16.9165 - val_loss: 0.0216 - val_psnr: 16.9355 - 1104s/epoch - 2s/step
Epoch 7/50
644/644 - 1338s - loss: 0.0217 - psnr: 16.9411 - val_loss: 0.0215 - val_psnr: 16.9623 - 1338s/epoch - 2s/step
Epoch 8/50
644/644 - 1418s - loss: 0.0215 - psnr: 16.9707 - val_loss: 0.0215 - val_psnr: 16.9700 - 1418s/epoch - 2s/step
Epoch 9/50
644/644 - 1412s - los

In [ ]:
evals = model.evaluate(
    test_gen,
    batch_size=GPU_BATCH_SIZE,
    verbose=2
)

In [ ]:
model.save('models/saved/diff_spec_denoiser_05122024/unet/unet_denoiser.keras')

### Evaluation, metrics, and visualising denoised output

In [ ]:
reload(utils)
fig = utils.get_psnr_and_loss_curves(history, together=True)
fig.savefig("models/saved/diff_spec_denoiser_05122024/unet/img/psnr_loss_curves.pdf")

In [ ]:
# Get a few samples from the test generator for visualisation and metrics calculation
num_samples = 5 

sample_X, sample_y = next(iter(test_gen))
sample_inputs = sample_X[:num_samples]  # Extract the first `num_samples` examples from the batch
sample_outputs = sample_y[:num_samples]  # Extract the first `num_samples` examples from the batch

# Predict the denoised outputs
denoised_outputs = model.predict(sample_inputs)

In [ ]:
print("Shape of sample_inputs:", sample_inputs.shape)
print("Shape of denoised_outputs:", denoised_outputs.shape)

In [ ]:
# Get evaluation metrics
mse_score = tf.keras.losses.MeanSquaredError()(sample_inputs, denoised_outputs).numpy()
psnr_score = utils.psnr(sample_inputs, denoised_outputs).numpy()

print(f"MSE on sample batch: {mse_score}")
print(f"PSNR on sample batch: {psnr_score}")

In [ ]:
# Check if all outputs are normalised
for i, output in enumerate(denoised_outputs):
    print(f"Sample {i+1} - Min: {np.min(output)}; Max: {np.max(output)}")

In [ ]:
# Create a single figure
fig, axes = plt.subplots(num_samples, 3, figsize=(14, num_samples * 3.5), sharex=True, sharey=True)

# fig.tight_layout(pad=3.0)

for i in range(num_samples):
    axes[i, 0].imshow(sample_inputs[i].squeeze(), cmap="viridis", aspect="auto")
    axes[i, 1].imshow(denoised_outputs[i].squeeze(), cmap="viridis", aspect="auto")
    axes[i, 2].imshow(sample_outputs[i].squeeze(), cmap="viridis", aspect="auto")

axes[0, 0].set_title(f"Input\n", fontsize=18)
axes[0, 1].set_title(f"Denoised Output\n", fontsize=18)
axes[0, 2].set_title(f"Output\n", fontsize=18)

fig.tight_layout(pad=1.5)
fig.savefig(f"models/saved/diff_spec_denoiser_05122024/unet/img/combined_spectrograms.pdf", bbox_inches="tight")

plt.show()

In [ ]:
# Plot the input, output, and target spectrograms for each sample
for i in range(num_samples):
    plt.figure(figsize=(12, 4))
    
    # INPUT
    plt.subplot(1, 3, 1)
    plt.imshow(sample_inputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Input")
    # plt.colorbar()

    # DENOISED OUTPUT
    plt.subplot(1, 3, 2)
    plt.imshow(denoised_outputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Denoised Output")
    # plt.colorbar()

    # TARGET
    plt.subplot(1, 3, 3)
    plt.imshow(sample_outputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Target")
    # plt.colorbar()
    
    plt.tight_layout()
    plt.show()